In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scinet import *
import scinet.ed_quantum as edq
%load_ext autoreload

## Train models

*Warning:* Training the required number of models takes a significant amount of time on a standard laptop. If you would like to download the pre-trained models and corresponding datasets instead (not included here because of large file size), you can do so [here](https://github.com/eth-nn-physics/quantum_data). To use these models, copy the contents of the downloaded `data` directory and `tf_save` directory into the corresponding directories of this project.

### Parameters
- `latent_size:` between 0 and 5
- `input_size: 10`
- `input2_size: 10`
- `output_size: 1`
- other parameters: default values
### Data
Both the qubit and the projection axis are specified by 10 random projective measurements.
- training data: 95000 samples
- validation data: 5000 samples

### Training
For each latent size, three training are performed and the one with the lowest error is chosen. Each training run is performed in two steps:
    1. `epoch_num: 250`, `batch_size: 512`, `learning_rate: 1e-3`, `beta: 1e-4`
    2. `epoch_num: 50`, `batch_size: 512`, `learning_rate: 1e-4`, `beta: 1e-4`
    
### Naming scheme
The models are named 'one_qubit_label_s_i', where label is 'comp' (for tomographically complete) or 'incomp' (for tomographically incomplete), 's' is the number of latent neurons and 'i' is the run (0 to 2). 

In [ ]:
# %autoreload
# for inc_tom_m1 in [2, False]:
#     label = 'incomp' if inc_tom_m1 else 'comp'
#     edq.create_data(1, 10, 10, 100000, 'one_qubit_{}'.format(label), incomplete_tomography=[inc_tom_m1, False])
#     td, vd, ts, vs, projectors = dl.load(5, 'one_qubit_{}'.format(label))
#     for s in range(5):
#         for i in range(3):
#             name = 'one_qubit_{}_{}_{}'.format(label, s, i)
#             net = nn.Network(10, s, 10, 1, name=name, encoder_num_units=[100, 100], decoder_num_units=[100, 100])
#             net.train(250, 512, 1e-3, td, vd, beta_fun=lambda x: 1.e-4, test_step=10)
#             net.train(50, 512, 1e-4, td, vd, beta_fun=lambda x: 1.e-4, test_step=10)
#             net.save(name)
#             ra = net.run(vd, net.recon_loss)
#             print ra

## Load pre-trained models

In [ ]:
err = []
for label in ['comp', 'incomp']:
    td, vd, ts, vs, projectors = dl.load(5, 'one_qubit_{}'.format(label))
    err_label = []
    for s in range(5):
        err_s = []
        for i in range(3):
            name = 'one_qubit_{}_{}_{}'.format(label, s, i)
            net = nn.Network.from_saved(name)
            print np.sqrt(net.run(vd, net.recon_loss))
            err_s.append(np.sqrt(net.run(vd, net.recon_loss)))
        err_label.append(min(err_s))
    err.append(err_label)

In [ ]:
latent_neurons = np.arange(5)
blue_color='#000cff'
orange_color='#ff7700'
fig = plt.figure(figsize=(3.4, 2.1))
ax = fig.add_subplot(111)
width = 0.32
ax.bar(latent_neurons - width/2., err[0], width=width, label='Tom. complete', color=blue_color)
ax.bar(latent_neurons + width/2., err[1], width=width, label='Tom. incomplete', color=orange_color)
ax.legend()
ax.set_xticks(latent_neurons)
ax.set_xlabel('Number of latent neurons')
ax.set_ylabel('Error of predictions')
fig.tight_layout()